<a href="https://colab.research.google.com/github/coywil26/DIMPLE/blob/master/DIMPLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Install

#@markdown Please execute each cell by pressing the *Play* button on the left.

# download github "https://github.com/coywil26/DIMPLE" and install dependencies
!git clone https://github.com/coywil26/DIMPLE

!pip install -r DIMPLE/requirements.txt

In [ ]:
#@title 2. Upload target file
#@markdown Please execute to upload file
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
target_file = fn
!mkdir workspace
!mv $target_file workspace
directory = "workspace/"

In [ ]:
#@title 3. Generate Mutation Fragments and Primers
#@markdown Oligo Parameters
from google.colab import files
import os
import argparse
from DIMPLE.DIMPLE.DIMPLE import align_genevariation, print_all, post_qc, addgene, DIMPLE, generate_DMS_fragments
from DIMPLE.DIMPLE.utilities import parse_custom_mutations, codon_usage
from Bio.Seq import Seq
from google.colab import data_table
import ast
import pandas as pd

class arguments:
  pass
args = arguments()

# if target_file doesnt exist use default test file
if 'target_file' not in locals():
  target_file = "DIMPLE/tests/combined_fasta.fa"
  args.wDir = "DIMPLE/tests/"
  print('test run')
else:
  #args.wDir = os.path.basename(target_file)
  args.wDir = directory

args.geneFile = target_file

args.oligoLen = 250 #@param {type:"integer"}
args.fragmentLen = "auto"  #@param {type:"string"}
if args.fragmentLen == "auto":
  args.fragmentLen = ""
args.overlap = 4 #@param {type:"integer"}
args.barcode_start = 0 #@param {type:"integer"}
type_IIS_restriction_enzyme = "BsmBI" #@param ["BsmBI", "BsaI"]
if type_IIS_restriction_enzyme == "BsmBI":
  args.restriction_sequence = "CGTCTC"
else:
  args.restriction_sequence = "GGTCTC"
sequences_to_avoid = "CGTCTC, GGTCTC" #@param {type:"string"}
args.avoid_sequence = sequences_to_avoid.split(", ")
args.matchSequences = False

#@markdown Codon Usage (If using custom usage you will see a button to upload file at the bottom)
args.usage = "human" #@param ["human", "ecoli"]
custom_codon_usage = False #@param {type:"boolean"}
# input custom mutation file
if custom_codon_usage:
  print("Look in DIMPLE/data/custom_codon_usage.txt for example file")
  uploaded = files.upload()
  usage_file = list(uploaded.keys())[0]
  with open(usage_file) as f:
    usage = f.readlines()
  args.usage = ast.literal_eval(usage[0].strip('\n'))
  pd.DataFrame(args.usage,index=[0])

#@markdown Select Mutations
select_domain_insertion = False #@param {type:"boolean"}
if select_domain_insertion:
  args.dis = True
else:
  args.dis = False
args.handle = "AGCGGGAGACCGGGGTCTCTGAGC" #@param {type:"string"}
select_insertions = False #@param {type:"boolean"}
insertions = "GGG,GGGGGG" #@param {type:"string"}
if select_insertions:
  args.insertions = insertions.split(",")
else:
  args.insertions = ""
select_deletions = False #@param {type:"boolean"}
deletions = "3,6" #@param {type:"string"}
if select_deletions:
  args.deletions = deletions.split(",")
else:
  args.deletions = ""
select_deep_mutation_scan = False #@param {type:"boolean"}
if select_deep_mutation_scan:
  args.dms = True
else:
  args.dms = False
amino_acid_substitutions = "Cys,Asp,Ser,Gln,Met,Asn,Pro,Lys,Thr,Phe,Ala,Gly,Ile,Leu,His,Arg,Trp,Val,Glu,Tyr" #@param {type:"string"}
args.include_stop_codons = False #@param {type:"boolean"}
args.include_synonymous_mutations = False #@param {type:"boolean"}
select_custom_mutations = False #@param {type:"boolean"}
if select_custom_mutations:
  print("Look in DIMPLE/data/custom_codon_usage.txt for example file")
  uploaded = files.upload()
  custom_mutations_file = list(uploaded.keys())[0]
  with open(custom_mutations_file) as f:
    lines = f.readlines()
  args.custom_mutations = parse_custom_mutations(lines[1:])
  pd.DataFrame(args.custom_mutations,index=[0])
else:
  args.custom_mutations = {}
args.make_double_mutations = False #@param {type:"boolean"}

#!python3 DIMPLE/run_dimple.py -geneFile=$target_file -oligoLen=$oligo_length -DMS=$select_DMS -usage=$codon_usage_bias -include_stop_codons=$include_stop_codons -restriction_sequence=$type_IIS_restriction_enzyme -avoid_sequence=$sequences_to_avoid -insertions=$insertions -deletions=$deletions -fragmentLen=$fragment_length -overlap=$fragment_overlap -barcode_start=$barcode_start_position -include_stop_codons=$include_stop_codons -usage=$custom_mutations


# Check genetic handle for non nucleic characters
if any([x not in ['A', 'C', 'G', 'T', 'a', 'c', 'g', 't'] for x in args.handle]):
    raise ValueError('Genetic handle contains non nucleic bases')

#DIMPLE.handle = args.handle
DIMPLE.synth_len = args.oligoLen
if args.fragmentLen:
    DIMPLE.maxfrag = args.fragmentLen
else:
    DIMPLE.maxfrag = args.oligoLen - 62 - args.overlap  # 62 allows for cutsites and barcodes

DIMPLE.dms = args.dms

#  adjust primer primerBuffer
DIMPLE.primerBuffer += args.overlap

DIMPLE.avoid_sequence = args.avoid_sequence
DIMPLE.barcodeF = DIMPLE.barcodeF[int(args.barcode_start):]
DIMPLE.barcodeR = DIMPLE.barcodeR[int(args.barcode_start):]
DIMPLE.cutsite = Seq(args.restriction_sequence)
DIMPLE.avoid_sequence = [Seq(x) for x in args.avoid_sequence]
DIMPLE.stop_codon = args.include_stop_codons
DIMPLE.make_double = args.make_double_mutations
DIMPLE.usage = codon_usage(args.usage)

OLS = addgene(os.path.join(args.wDir, args.geneFile).strip())

if args.matchSequences == 'match':
    align_genevariation(OLS)
if args.deletions:
    args.deletions = [int(x) for x in args.deletions]
if not any([args.dms, args.insertions, args.deletions]):
    raise ValueError("Didn't select any mutations to generate")
print(args.dms)
print(args.overlap)
generate_DMS_fragments(OLS, args.overlap, args.overlap, args.include_synonymous_mutations, args.custom_mutations, args.dms, args.insertions, args.deletions, args.dis, args.wDir)

post_qc(OLS)
print_all(OLS, args.wDir)

directory = args.wDir
!zip -r file.zip $directory
files.download("file.zip")